In [3]:
# Suprimir advertencias
import warnings
warnings.filterwarnings("ignore")

#Importing Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import datetime
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency

In [4]:
#Loading Data:
data = pd.read_csv("Data_PCRIFPTML-model.csv", sep=';')

In [205]:
# CHECKING DATA
data["prediction_trainig"] = data["prediction_trainig"].str.strip()
print(len(data[data["prediction_trainig"] == "training"]))
print(len(data[data["prediction_trainig"] == "validation"]))

436
131


In [207]:
#Data Splitting:
Y = data.iloc[:, 1]
X = data.iloc[:, 3:]

In [208]:
#Splitting Data into Training and Prediction Sets:
train_data = data[data["prediction_trainig"] == "training"]
pred_data = data[data["prediction_trainig"] == "validation"]

In [210]:
# Model Preparation for LDA
X_train_LDA= train_data.iloc[:, 3:]
X_test_LDA= pred_data.iloc[:, 3:]
y_train_LDA= train_data.iloc[:, 1]
y_test_LDA= pred_data.iloc[:, 1]

print(X_train_LDA.shape)
print(X_test_LDA.shape)
print(y_train_LDA.shape)
print(y_test_LDA.shape)

(436, 10)
(131, 10)
(436,)
(131,)


In [212]:
#X_LDA
mlr_X_train_LDA= X_train_LDA[['fref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIIIcIII', 'sIIIcvII','sIVcIV','sIVcIV','sIVcIV','sVIcvI']]
mlr_X_test_LDA= X_test_LDA[['fref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIIIcIII', 'sIIIcvII','sIVcIV','sIVcIV','sIVcIV','sVIcvI']]

In [213]:
#Model_LDA

In [215]:
# Define the Prior Classification Probabilities
priors = [0.5, 0.5]  

In [218]:
# Initialize the LDA model with the prior probabilities
LDA = LinearDiscriminantAnalysis(priors=priors)

In [219]:
#Fit the model

In [221]:
LDA.fit(mlr_X_train_LDA, y_train_LDA)

LinearDiscriminantAnalysis(priors=[0.5, 0.5])

In [222]:
y_pred_train_LDA = LDA.predict(mlr_X_train_LDA)
y_pred_test_LDA = LDA.predict(mlr_X_test_LDA)

In [224]:
#Perform cross-validation with LinearDiscriminantAnalysis
cv_scores = cross_val_score(LDA, mlr_X_train_LDA, y_train_LDA, cv=5)  # 5-fold cross-validation
print("Cross-validation Scores:", cv_scores)

Cross-validation Scores: [0.48863636 0.73563218 0.82758621 0.82758621 0.89655172]


In [226]:
# Get variable names
variable_names = X.columns

In [229]:
# Coefficients of the linear discriminant function
coefficients = LDA.coef_[0]
intercept = LDA.intercept_[0]

In [251]:
# Imprimir la ecuación del modelo de forma segura
print("Equation of the line:")
print("y = ", end="")

# zip() se detiene en el tamaño más corto entre las dos listas, evitando errores de índice
for coefficient, variable_name in zip(coefficients, variable_names):
    print(f"({coefficient:.3f} * {variable_name}) + ", end="")

# Imprimir la constante del modelo
print(f"{intercept:.3f}")

Equation of the line:
y = (12.296 * fref) + (-0.902 * sIcI) + (3.865 * sIcIV) + (-1.871 * sIIcIV) + (-0.513 * sIIIcII) + (0.034 * sIIIcIII) + (-0.400 * sIIIcvII) + (0.531 * sIVcIV) + (0.531 * sVcv) + (0.531 * sVIcvI) + -4.492


In [232]:
confusion_train = confusion_matrix(y_train_LDA,y_pred_train_LDA)
confusion_test = confusion_matrix(y_test_LDA, y_pred_test_LDA)

In [233]:
y_train_pred_proba = LDA.predict_proba(mlr_X_train_LDA)
y_test_pred_proba = LDA.predict_proba(mlr_X_test_LDA)

In [235]:
print('Confusion Matrix for Training Series')
print(confusion_train)
print('Confusion Matrix for Test Series')
print(confusion_test)

Confusion Matrix for Training Series
[[280  67]
 [ 19  70]]
Confusion Matrix for Test Series
[[89 23]
 [ 3 16]]


In [236]:
tn, fp, fn, tp = confusion_matrix(y_train_LDA, y_pred_train_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [239]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy)

Sp(%)= 0.8069164265129684 Sn(%)= 0.7865168539325843 Ac(%)= 0.8027522935779816


In [240]:
tn, fp, fn, tp = confusion_matrix(y_test_LDA, y_pred_test_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [242]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy),

Sp(%)= 0.7946428571428571 Sn(%)= 0.8421052631578947 Ac(%)= 0.8015267175572519


(None,)